# **Práctica 10: Archivos.**

## **Librerías**

In [15]:
import os
import json
import random
import string
from datetime import datetime
import matplotlib.pyplot as plt

## **Clases de Modelos**

In [16]:
class User:
    def __init__(self, user_id, name, password):
        self.id = user_id
        self.name = name
        self.password = password
    
    def __str__(self):
        return f"User(ID={self.id}, Nombre={self.name})"

class File:
    def __init__(self, file_id, level_access, name, owner, content=""):
        self.id = file_id
        self.level_access = level_access
        self.name = name
        self.owner = owner
        self.content = content
    
    def __str__(self):
        return f"File(ID={self.id}, Nombre={self.name}, owner={self.owner})"

class Session:
    def __init__(self, user_id, isValid=True):
        self.user_id = user_id
        self.datetime = datetime.now()
        self.isValid = isValid
    
    def __str__(self):
        return f"Session(UserID={self.user_id}, Fecha={self.datetime}, Válido={self.isValid})"

## **Árbol B**

In [17]:
class Data:
    def __init__(self, key, value):
        self.key = key
        self.value = value

    def __str__(self):
        return str(self.key) + "->" + self.value

class BTreeNode:
    def __init__(self, t, leaf):
        self.t = t
        self.keys = [None] * (2 * t - 1)
        self.C = [None] * (2 * t)
        self.n = 0
        self.leaf = leaf

    def insertNonFull(self, data):
        i = self.n - 1
        if self.leaf:
            while i >= 0 and self.keys[i].key > data.key:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = data
            self.n += 1
        else:
            while i >= 0 and self.keys[i].key > data.key:
                i -= 1
            if self.C[i + 1].n == 2 * self.t - 1:
                self.splitChild(i + 1, self.C[i + 1])
                if self.keys[i + 1].key < data.key:
                    i += 1
            self.C[i + 1].insertNonFull(data)

    def splitChild(self, i, y):
        z = BTreeNode(y.t, y.leaf)
        z.n = self.t - 1
        for j in range(self.t - 1):
            z.keys[j] = y.keys[j + self.t]
        if not y.leaf:
            for j in range(self.t):
                z.C[j] = y.C[j + self.t]
        y.n = self.t - 1
        for j in range(self.n, i, -1):
            self.C[j + 1] = self.C[j]
        self.C[i + 1] = z
        for j in range(self.n - 1, i - 1, -1):
            self.keys[j + 1] = self.keys[j]
        self.keys[i] = y.keys[self.t - 1]
        self.n += 1

    def traverse(self, l):
        for i in range(self.n):
            if not self.leaf:
                self.C[i].traverse(l + 1)
            print("\t" * l, l, self.keys[i], end=' ')
        print()
        if not self.leaf:
            # Corregido: El PDF decía self.C[i + 1], lo cual es un error.
            # Debe ser el último hijo, self.C[self.n].
            self.C[self.n].traverse(l + 1)

    def search(self, k):
        i = 0
        while i < self.n and k > self.keys[i].key:
            i += 1
        if i < self.n and k == self.keys[i].key:
            return self
        if self.leaf:
            return None
        return self.C[i].search(k)

class BTree:
    def __init__(self, t):
        self.root = None
        self.t = t

    def traverse(self):
        if self.root != None:
            self.root.traverse(0)

    def search(self, k):
        return None if self.root == None else self.root.search(k)

    def insert(self, data):
        if self.root == None:
            self.root = BTreeNode(self.t, True)
            self.root.keys[0] = data
            self.root.n = 1
        else:
            if self.root.n == 2 * self.t - 1:
                s = BTreeNode(self.t, False)
                s.C[0] = self.root
                s.splitChild(0, self.root)
                i = 0
                if s.keys[0].key < data.key:
                    i += 1
                s.C[i].insertNonFull(data)
                self.root = s
            else:
                self.root.insertNonFull(data)


## **Funciones para Gestionar los JSON**

In [18]:
class JsonController:
    def __init__(self):
        self.files_tree = BTree(3)
        self.sessions = []
        self.file_counters = {}
        self._cargar_sesiones()
        self.cargar_archivos_desde_json()
    
    def _cargar_sesiones(self):
        """Cargar sesiones existentes desde sessions.json"""
        try:
            if os.path.exists("sessions.json"):
                with open("sessions.json", "r", encoding="utf-8") as file:
                    sessions_data = json.load(file)
                    for session_data in sessions_data:
                        session = Session(
                            session_data["user_id"],
                            session_data["isValid"]
                        )
                        session.datetime = datetime.fromisoformat(session_data["datetime"])
                        self.sessions.append(session)
        except Exception as e:
            print(f"Error al cargar sesiones: {e}")
    
    def cargar_archivos_desde_json(self):
        """Cargar archivos existentes desde files.json al árbol B"""
        try:
            if os.path.exists("files.json"):
                with open("files.json", "r", encoding="utf-8") as file:
                    files_data = json.load(file)
                    for file_data in files_data:
                        nuevo_archivo = File(str(file_data["id"]), file_data["level_access"], file_data["name"], file_data["owner"], file_data["content"])
                        file_data_obj = Data(nuevo_archivo.id, nuevo_archivo)
                        self.files_tree.insert(file_data_obj)
                print(f"Se cargaron {len(files_data)} archivos al árbol B")
        except Exception as e:
            print(f"Error al cargar archivos: {e}")
    
    def verificar_usuario(self, user_id, password):
        """Verificar el ID y password de user desde users.json"""
        try:
            if not os.path.exists("users.json"):
                print("Error: Archivo users.json no encontrado")
                return None
            
            with open("users.json", "r", encoding="utf-8") as file:
                users_data = json.load(file)
            
            for user_data in users_data:
                if str(user_data.get("id")) == user_id and user_data.get("password") == password:
                    return User(user_id, user_data["name"], password)
            
            return None
            
        except Exception as e:
            print(f"Error al leer users.json: {e}")
            return None
    
    def _guardar_sesiones_json(self):
        """Guardar sesiones en sessions.json"""
        try:
            sessions_data = []
            for session in self.sessions:
                session_data = {
                    "user_id": session.user_id,  # Cambiado de ID_del_user
                    "datetime": session.datetime.isoformat(),  # Cambiado de Fecha_y_Hora
                    "isValid": session.isValid  # Cambiado de Ingreso_Válido
                }
                sessions_data.append(session_data)
            
            with open("sessions.json", "w", encoding="utf-8") as file:
                json.dump(sessions_data, file, indent=2)
        except Exception as e:
            print(f"Error al guardar sesiones: {e}")
    
    def _actualizar_files_json(self):
        """Actualizar files.json con todos los archivos del árbol B"""
        try:
            archivos_list = []
            self._recopilar_archivos(self.files_tree.root, archivos_list)
            
            files_data = []
            for archivo in archivos_list:
                file_data = {
                    "id": int(archivo.id),  # Cambiado de ID a id
                    "level_access": archivo.level_access,  # Cambiado de Nivel_de_Acceso
                    "name": archivo.name,  # Cambiado de Nombre a name
                    "owner": archivo.owner,  # Cambiado de Propietario
                    "content": archivo.content  # Cambiado de Contenido
                }
                files_data.append(file_data)
            
            with open("files.json", "w", encoding="utf-8") as file:
                json.dump(files_data, file, indent=2)
                
            print("Archivo files.json actualizado")
            
        except Exception as e:
            print(f"Error al actualizar files.json: {e}")
    
    def guardar_sesion(self, session):
        """Guardar los registros de session"""
        self.sessions.append(session)
        self._guardar_sesiones_json()
    
    def guardar_archivo(self, file):
        """Guardar archivo en el árbol B y actualizar JSON"""
        file_data = Data(file.id, file)  # Cambiado de ID a id
        self.files_tree.insert(file_data)
        self._actualizar_files_json()
    
    def cargar_archivo(self, file_id):
        """Cargar archivo del árbol B"""
        file_data = self.files_tree.search(file_id)
        return file_data.data if file_data else None
    
    def generar_file_id(self, nivel_acceso):
        """Generar ID único: nivel_acceso + número autoincrementable de 4 dígitos"""
        max_num = 0
        archivos_list = []
        self._recopilar_archivos(self.files_tree.root, archivos_list)
        
        for archivo in archivos_list:
            file_id_str = archivo.id  # Cambiado de ID a id
            if file_id_str.startswith(str(nivel_acceso)):
                if len(file_id_str) > len(str(nivel_acceso)):
                    num_part = file_id_str[len(str(nivel_acceso)):]
                    try:
                        current_num = int(num_part)
                        max_num = max(max_num, current_num)
                    except ValueError:
                        continue
        
        next_num = max_num + 1
        file_id = f"{nivel_acceso}{next_num:04d}"
        return file_id
    
    def _recopilar_archivos(self, node, archivos_list, exclude_id=None):
        """Recopilar todos los archivos del árbol B"""
        if node is None:
            return
        
        for i in range(node.n):
            if not node.leaf:
                self._recopilar_archivos(node.C[i], archivos_list, exclude_id)
            
            archivo = node.keys[i].value
            if exclude_id is None or archivo.id != exclude_id:  # Cambiado de ID a id
                archivos_list.append(archivo)
        
        if not node.leaf:
            self._recopilar_archivos(node.C[node.n], archivos_list, exclude_id)
    
    def obtener_sesiones_usuario(self, user_id):
        """Obtener todas las sesiones de un usuario"""
        return [session for session in self.sessions if session.user_id == user_id]  # Cambiado de ID_del_user

## **Funciones para Graficar**

## **Clase Principal**

In [ ]:
class FileSystem:
    def __init__(self):
        self.json_controller = JsonController()
        self.current_user = None
        self.current_session = None
    
    def iniciar_sesion(self, user_id, password):
        """Iniciar Sesión"""
        usuario = self.json_controller.verificar_usuario(user_id, password)
        if usuario:
            self.current_user = usuario
            self.current_session = Session(user_id, True)
            self.json_controller.guardar_sesion(self.current_session)
            print(f"¡Bienvenido {usuario.name}!")  # Cambiado de Nombre a name
            return True
        else:
            session_fallida = Session(user_id, False)
            self.json_controller.guardar_sesion(session_fallida)
            print("Error: ID de usuario o contraseña incorrectos")
            return False
    
    def crear_archivo(self, name, level_access, content=""):
        """Crear Archivo"""
        if not self.current_user:
            print("Error: Debe iniciar sesión primero")
            return None
        
        file_id = self.json_controller.generar_file_id(level_access)
        nuevo_archivo = File(file_id, level_access, name, self.current_user.id, content)  # Cambiado de ID a id
        self.json_controller.guardar_archivo(nuevo_archivo)
        print(f"Archivo creado: {nuevo_archivo}")
        return nuevo_archivo
    
    def buscar_abrir_archivo(self, file_name):
        """
        Busca un archivo por su nombre y muestra la información
        basado en el nivel de acceso del usuario logueado.
        """
        # 1. VERIFICAR SESIÓN
        if not self.usuario_actual:
            print("Error: Debe iniciar sesión primero.")
            return

        usuario_id = self.usuario_actual['user_id']
        usuario_level = self.usuario_actual['level_access']
        
        # 2. BUSCAR ARCHIVO POR NOMBRE
        # LLAMADA AL NUEVO MÉTODO DEL CONTROLADOR (Ver sección 2)
        archivo_encontrado = self.json_controller.buscar_archivo_por_nombre(file_name)

        if not archivo_encontrado:
            print(f"Error: Archivo '{file_name}' no encontrado.")
            return

        # 3. VERIFICAR ACCESO Y MOSTRAR INFORMACIÓN
        acceso_requerido = archivo_encontrado['level_access']

        print(f"Archivo: {archivo_encontrado['name']} (ID: {archivo_encontrado['file_id']})")
        print(f"Nivel de Acceso Requerido: {acceso_requerido}")

        if usuario_level >= acceso_requerido:
            # Si el nivel del usuario es igual o superior al requerido, tiene acceso.
            print("¡ACCESO CONCEDIDO! Mostrando información completa del archivo:")
            # Mostrar la metadata completa o el contenido del archivo
            for key, value in archivo_encontrado.items():
                print(f"  - {key}: {value}")
        else:
            # Acceso denegado. Se aplica la restricción según el nivel requerido:
            print(f"¡ACCESO DENEGADO! Su nivel de acceso ({usuario_level}) es insuficiente.")
            
            # Lógica de restricción basada en la tabla (imagen)
            if acceso_requerido == 1:
                # Nivel 1 (Público): Muestra solo ID, Name, Level.
                print("Mostrando información parcial (Público):")
                print(f"  - file_id: {archivo_encontrado['file_id']}")
                print(f"  - name: {archivo_encontrado['name']}")
                print(f"  - level_access: {archivo_encontrado['level_access']}")

            elif acceso_requerido == 2:
                # Nivel 2 (Autor): Muestra ID, Name, Level.
                print("Mostrando información parcial (Autor):")
                print(f"  - file_id: {archivo_encontrado['file_id']}")
                print(f"  - name: {archivo_encontrado['name']}")
                print(f"  - level_access: {archivo_encontrado['level_access']}")
                
            elif acceso_requerido == 3:
                # Nivel 3 (Administrador): Muestra solo ID y Name.
                print("Mostrando información parcial (Administrador):")
                print(f"  - file_id: {archivo_encontrado['file_id']}")
                print(f"  - name: {archivo_encontrado['name']}")

            else:
                # Para cualquier otro nivel no definido, no mostrar nada o un mensaje genérico.
                print("No se puede mostrar información parcial para este nivel de acceso.")
    
    def modificar_archivo(self, file_id, nuevo_contenido):
        """Modificar Archivo"""
        if not self.current_user:
            print("Error: Debe iniciar sesión primero")
            return False
        
        archivo = self.json_controller.cargar_archivo(file_id)
        if archivo:
            if archivo.owner == self.current_user.id:  # Cambiado de Propietario a owner y ID a id
                success = self.json_controller.modificar_archivo(file_id, nuevo_contenido)
                if success:
                    print(f"Archivo modificado: {archivo.name}")  # Cambiado de Nombre a name
                    print(f"Nuevo content: {nuevo_contenido}")  # Cambiado de contenido a content
                return success
            else:
                print("Error: Solo el owner puede modificar el archivo")  # Cambiado de propietario a owner
                return False
        else:
            print("Error: Archivo no encontrado")
            return False
    
    def eliminar_archivo(self, file_id):
        """Eliminar Archivo"""
        if not self.current_user:
            print("Error: Debe iniciar sesión primero")
            return False
        
        archivo = self.json_controller.cargar_archivo(file_id)
        if archivo:
            if archivo.owner == self.current_user.id:  # Cambiado de Propietario a owner y ID a id
                self.json_controller.eliminar_archivo(file_id)
                print(f"Archivo eliminado: {archivo.name}")  # Cambiado de Nombre a name
                return True
            else:
                print("Error: Solo el owner puede eliminar el archivo")  # Cambiado de propietario a owner
                return False
        else:
            print("Error: Archivo no encontrado")
            return False
    
    def listar_archivos(self):
        """Listar Archivos"""
        if not self.current_user:
            print("Error: Debe iniciar sesión primero")
            return
        
        print("Lista de archivos:")
        archivos_list = []
        self.json_controller._recopilar_archivos(self.json_controller.files_tree.root, archivos_list)
        
        if not archivos_list:
            print("No hay archivos en el sistema")
            return
            
        for archivo in archivos_list:
            print(f"- ID: {archivo.id}, Name: {archivo.name}, Owner: {archivo.owner}")  # Cambiados los nombres
    
    def listar_archivos_propietario(self):
        """Listar archivos del usuario actual"""
        if not self.current_user:
            print("Error: Debe iniciar sesión primero")
            return
        
        archivos = self.obtener_archivos_propietario()
        if not archivos:
            print("No tienes archivos en el sistema")
            return
        
        print(f"\nTus archivos ({len(archivos)}):")
        for archivo in archivos:
            print(f"- ID: {archivo.id}, Name: {archivo.name}, Level Access: {archivo.level_access}")  # Cambiados los nombres
    
    def obtener_archivos_propietario(self):
        """Obtener archivos del usuario actual"""
        if not self.current_user:
            print("Error: Debe iniciar sesión primero")
            return []
        
        archivos_propietario = []
        archivos_list = []
        self.json_controller._recopilar_archivos(self.json_controller.files_tree.root, archivos_list)
        
        for archivo in archivos_list:
            if archivo.owner == self.current_user.id:  # Cambiado de Propietario a owner y ID a id
                archivos_propietario.append(archivo)
        
        return archivos_propietario
    
    def ver_historial_sesiones(self, user_id=None):
        """Ver historial de sesiones"""
        if user_id is None:
            if self.current_user:
                user_id = self.current_user.id  # Cambiado de ID a id
            else:
                print("Error: Debe especificar un user_id o tener sesión activa")
                return
        
        sesiones = self.json_controller.obtener_sesiones_usuario(user_id)
        print(f"\nHistorial de sesiones para usuario {user_id}:")
        for i, session in enumerate(sesiones, 1):
            print(f"{i}. {session}")
    
    def cerrar_sesion(self):
        """Cerrar Sesión"""
        if self.current_user:
            print(f"Sesión cerrada para {self.current_user.name}")  # Cambiado de Nombre a name
            self.current_user = None
            self.current_session = None
        else:
            print("No hay sesión activa")

## **Menú**

## **Ejemplo de Uso**

In [23]:
# Ejemplo de uso completo del FileSystem
if __name__ == "__main__":
    # Crear sistema de archivos
    print("=== INICIALIZANDO SISTEMA DE ARCHIVOS ===")
    fs = FileSystem()
    
    # # 1. Intentar iniciar sesión con credenciales incorrectas
    # print("\n=== 1. INTENTO DE LOGIN FALLIDO ===")
    # fs.iniciar_sesion("1", "password_incorrecto")
    
    # 2. Iniciar sesión correctamente
    print("\n=== 2. LOGIN EXITOSO ===")
    fs.iniciar_sesion("1", "qG3%{h'bK")  # Usuario Dion
    
    # 3. Listar archivos existentes en el sistema
    # print("\n=== 3. LISTAR TODOS LOS ARCHIVOS ===")
    # fs.listar_archivos()
    
    # # 4. Listar archivos del propietario actual
    # print("\n=== 4. MIS ARCHIVOS ===")
    # fs.listar_archivos_propietario()
    
    # 5. Buscar y abrir un archivo existente
    # print("\n=== 5. BUSCAR ARCHIVO EXISTENTE ===")      # CHECAR PORQUE EN EL ARBOL SE GUARDA SOLO EL ID NO LA INFO
    # fs.buscar_abrir_archivo("10001")  # Archivo con ID 10001
    
    # # 6. Crear nuevos archivos
    # print("\n=== 6. CREAR NUEVOS ARCHIVOS ===")
    # archivo1 = fs.crear_archivo("mi_documento.txt", 1, "Este es mi primer documento")
    # archivo2 = fs.crear_archivo("datos_importantes.csv", 3, "nombre,edad,ciudad\nJuan,25,Lima\nMaria,30,Bogota")
    
    # # 7. Listar archivos después de crear nuevos
    # print("\n=== 7. ARCHIVOS DESPUÉS DE CREAR NUEVOS ===")
    # fs.listar_archivos()
    
    # # 8. Modificar un archivo propio
    # print("\n=== 8. MODIFICAR ARCHIVO PROPIO ===")
    # if archivo1:
    #     fs.modificar_archivo(archivo1.ID, "Este es mi primer documento - VERSIÓN MODIFICADA")
    
    # # 9. Intentar modificar archivo de otro usuario
    # print("\n=== 9. INTENTAR MODIFICAR ARCHIVO AJENO ===")
    # fs.modificar_archivo("2", "Este contenido no debería poder modificarse")
    
    # # 10. Buscar el archivo modificado
    # print("\n=== 10. VER ARCHIVO MODIFICADO ===")
    # if archivo1:
    #     fs.buscar_abrir_archivo(archivo1.ID)
    
    # # 11. Eliminar un archivo propio
    # print("\n=== 11. ELIMINAR ARCHIVO PROPIO ===")
    # if archivo2:
    #     fs.eliminar_archivo(archivo2.ID)
    
    # # 12. Intentar eliminar archivo de otro usuario
    # print("\n=== 12. INTENTAR ELIMINAR ARCHIVO AJENO ===")
    # fs.eliminar_archivo("3")  # Archivo de Hallie
    
    # # 13. Ver historial de sesiones del usuario actual
    # print("\n=== 13. HISTORIAL DE SESIONES ===")
    # fs.ver_historial_sesiones()
    
    # # 14. Crear más archivos para demostrar el sistema de IDs
    # print("\n=== 14. CREAR ARCHIVOS CON DIFERENTES NIVELES DE ACCESO ===")
    # archivo3 = fs.crear_archivo("publico.txt", 1, "Contenido público")
    # archivo4 = fs.crear_archivo("privado.txt", 1, "Contenido privado")  # Mismo nivel, siguiente ID
    # archivo5 = fs.crear_archivo("secreto.txt", 5, "Contenido secreto")  # Diferente nivel
    
    # # 15. Listar archivos finales
    # print("\n=== 15. ESTADO FINAL DE ARCHIVOS ===")
    # fs.listar_archivos()
    
    # # 16. Cerrar sesión
    # print("\n=== 16. CERRAR SESIÓN ===")
    # fs.cerrar_sesion()
    
    # # 17. Intentar operaciones sin sesión
    # print("\n=== 17. OPERACIONES SIN SESIÓN ===")
    # fs.crear_archivo("sin_sesion.txt", 1, "Este no debería crearse")
    
    # # 18. Iniciar sesión con otro usuario
    # print("\n=== 18. INICIAR SESIÓN CON OTRO USUARIO ===")
    # fs.iniciar_sesion("3", "kT6*{(O\\LH3&D")  # Usuario Reta
    
    # # 19. Ver archivos del nuevo usuario
    # print("\n=== 19. ARCHIVOS DEL NUEVO USUARIO ===")
    # fs.listar_archivos_propietario()
    
    # # 20. Cerrar sesión final
    # print("\n=== 20. CERRAR SESIÓN FINAL ===")
    # fs.cerrar_sesion()
    
    # print("\n=== DEMOSTRACIÓN COMPLETADA ===")

=== INICIALIZANDO SISTEMA DE ARCHIVOS ===
Se cargaron 1000 archivos al árbol B

=== 2. LOGIN EXITOSO ===
¡Bienvenido Dion!
